In [ ]:
!pip install iminizinc

In [ ]:
import os
import numpy as np
import pandas as pd
%load_ext iminizinc

In [ ]:
classes = 6
rooms = 1
sections = 2
teachers = 3

### Randomly initialization

In [ ]:
tList = np.random.randint(1,teachers+1,size=(classes))
sList = np.random.randint(1,sections+1,size=(classes))
hList = np.random.randint(3,4,size=(classes))

tList[0] = 1
j=0
for i in range(0,classes):
    if i%sections==0:
#         print("i ran",tList[i])
        j=j+1
        for s in range(0,sections):
            tList[i+s] = j
            sList[i+s] = s+1

classDetails = np.array([tList, sList, hList])
classDetails = classDetails.T
# print(classDetails)
# print(classDetails.shape[0])
print(classDetails)
classes = classDetails.shape[0]
classDetails = classDetails.tolist()

In [ ]:
tBusy = np.zeros((teachers,5,6))
for i in range(0,6):
    tBusy[0,0,i] = 1
    tBusy[0,1,i] = 1
    tBusy[1,0,i] = 1
# print(tBusy.astype(int))
teacherBusy = tBusy.astype(int).tolist()

### Minizinc Model

In [ ]:
%%minizinc -m bind

include "globals.mzn";
set of int: slotsPerDay = 1..6;
set of int: daysPerWeek = 1..5;

set of int: properties = 1..3; 
%property0 = teacher
%property1 = section
%property2 = hours per week

int: classes;
int: rooms;
int: sections;
int: teachers;

set of int: sectionList = 1..sections;
set of int: teacherList = 1..teachers;
set of int: roomList = 1..rooms;
set of int: classList = 1..classes;
set of int: classIndex = 0..classes;

array[classList,properties] of int: classDetails;

array[teacherList,daysPerWeek,slotsPerDay] of var classIndex: teacherRoutine;
array[teacherList,daysPerWeek,slotsPerDay] of 0..1: teacherBusy;
array[sectionList,daysPerWeek,slotsPerDay] of var classIndex: sectionRoutine;
array[roomList,daysPerWeek,slotsPerDay] of var classIndex: roomRoutine;

% a class will not be repeated in the same day
constraint 
forall(i in classList, d in daysPerWeek)
    (
        count(teacherRoutine[classDetails[i,1],d,..],i)<=1
    );

% total hours of each course will be equal to the credit hour
constraint 
forall(i in classList)
    (
        sum(d in daysPerWeek)(count(teacherRoutine[classDetails[i,1],d,..],i))==classDetails[i,3]
    );

% matching teacher's routine with students'
constraint 
forall(i in classList,d in daysPerWeek, s in slotsPerDay)
    (
        if teacherRoutine[classDetails[i,1],d,s] == i 
        then sectionRoutine[classDetails[i,2],d,s] == i 
        else true 
        endif
    );

% considering teacher's availability
constraint
forall(t in teacherList, d in daysPerWeek, s in slotsPerDay)
    (
        if teacherBusy[t,d,s] == 1
        then teacherRoutine[t,d,s] = 0
        else true
        endif
    );


solve satisfy;

### Print Output

In [ ]:
print(np.array(sectionRoutine))

In [ ]:
print(np.array(teacherRoutine))

In [ ]:
# np.array(teacherRoutine).shape